# Transcription using OpenAi whisper. 

In [13]:
import numpy as np
from collections import Counter
import librosa
import numpy as np
import IPython.display
import IPython
from torchaudio.utils import download_asset
from IPython.display import Audio
import soundfile as sf
import csv
import os
import pandas as pd
import glob
pd.set_option('display.max_rows', None)


In [14]:
pip install openai


Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -U openai-whisper


  Using cached tiktoken-0.3.1-cp310-cp310-win_amd64.whl (581 kB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.3.3
    Uninstalling tiktoken-0.3.3:
      Successfully uninstalled tiktoken-0.3.3


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\vip\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~-ktoken\\_tiktoken.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [16]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

  Cloning https://github.com/openai/whisper.git to c:\users\vip\appdata\local\temp\pip-req-build-ehth5ca5
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tiktoken-0.3.3-cp310-cp310-win_amd64.whl (579 kB)
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.3.1
    Uninstalling tiktoken-0.3.1:
      Successfully uninstalled tiktoken-0.3.1


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\vip\AppData\Local\Temp\pip-req-build-ehth5ca5'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-whisper 20230314 requires tiktoken==0.3.1, but you have tiktoken 0.3.3 which is incompatible.


In [17]:
pip install flair

Note: you may need to restart the kernel to use updated packages.


In [18]:
import whisper
model = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."

)

Model is English-only and has 71,825,408 parameters.


the output of transcription of whisper asr model, is a dictionary of **text**, **segments**,**language**

**segments** : refers to the segments or portions of the audio that have been transcribed. Each segment typically represents a specific section of the input audio along with its corresponding transcribed text
**the structure of each segment** : includes attributes,
1. id: The ID assigned to the segment. It provides a unique identifier for each segment in the transcription output.
2. seek: The seek value represents the position of the segment within the audio file. It specifies the starting point of the segment, typically in seconds.
3. start: The start time of the segment, indicating the beginning timestamp of the transcribed portion within the audio file.
4. end: The end time of the segment, representing the timestamp when the transcribed portion ends within the audio file.
5. text: The transcribed text for the corresponding segment. It contains the recognized speech for that specific portion of the audio.
6. **tokens:** The tokens attribute represents the transcribed text segmented **into** individual units, often **words**. It provides a list of these tokens for the given segment, Each of these units is then assigned a unique identifier or index. These identifiers allow for efficient representation and processing of the text during model training and inference, These numbers serve as references to specific words or subword units within the transcribed text
7. **temperature**: The temperature value indicates the level of **randomness** or **diversity** in the model's generation process. A higher temperature generally results in more diverse output, while a lower temperature yields more focused and deterministic output.
8. **avg_logprob**: The average log probability of the segment indicates **the model's confidence or certainty for the transcribed text in that particular segment**. Higher values typically indicate higher confidence.
9. **compression_ratio**: The compression ratio represents the ratio of the original audio duration to the duration of the transcribed segment. It provides information about the **efficiency of the model's transcription**, where a higher value suggests more compression.
10. **no_speech_prob**: The probability of no speech occurring in the segment. It represents the **likelihood of there being no speech or silence** within the transcribed portion.



In [19]:
#Calculate lexical diversity, TTR
def calculate_ttr(transcription):
    words = transcription.split()
    #print (words)
    num_words = len(words)
    unique_words = len(set(words))
    ttr = unique_words / num_words
    return ttr

In [20]:
#Simpson's Diversity Index (SDI) for lexical diversity which considers both the number of unique words and their frequency of occurrence.
def calculate_sdi(transcription):
    words = transcription.split()
    word_counts = Counter(words)
    #print(word_counts)
    num_words = len(words)
    sdi = 1 - sum((count / num_words) ** 2 for count in word_counts.values())
    return sdi

In [21]:
#pauses 
def detect_pauses(audio_file, sr, min_pause_duration=0.5):
    # Load audio file
    audio, _ = librosa.load(audio_file, sr=sr)
    
    # Convert audio to mono if it has multiple channels
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    
    # Apply voice activity detection (VAD) or other methods to segment speech
    
    # Example using librosa VAD
    speech_segments = librosa.effects.split(audio, top_db=20)
    
    # Calculate pause durations between speech segments
    #sr = bundle.sample_rate
    pause_durations = []
    pauses = []
    prev_end = 0.0
    for start, end in speech_segments:
        pause_duration = start - prev_end
        if pause_duration >= min_pause_duration:
            pauses.append(pause_duration)
        prev_end = end
    for pause in pauses:
        pause_duration = pause / sr
        pause_durations.append(pause_duration)
        
    total_duration = len(audio_file) / sr
    pause_rate = sum(pause_durations) / total_duration
    return pause_rate 


In [22]:
from flair.models import SequenceTagger
from flair.data import Sentence

def calculate_syntactic_complexity(transcription):
    words = transcription.split()
    sentence = ' '.join(words)
    total_clauses = 0
    num_words = len(words) 
    average_sentence_length = num_words / len(sentence)
    #print(average_sentence_length)
    
    tagger = SequenceTagger.load('pos')
    line = Sentence(transcription)
    tagger.predict(line)
    pos_tags = [(token.text, token.labels[0].value) for token in line]
    
    #print(pos_tags)
    num_clauses = 0
    for i in range(len(pos_tags)-1):
        current_tag = pos_tags[i][1]
        next_tag = pos_tags[i+1][1]
        if current_tag.startswith('V') and next_tag not in ['.', ':', 'IN']:
            num_clauses += 1
    total_clauses += num_clauses
    average_clauses_per_sentence = total_clauses / num_words
    
    return average_sentence_length, average_clauses_per_sentence

In [ ]:
# # Create and open the CSV file for writing
# audio_directories = ["/kaggle/input/adresso-audio-data/AD", "/kaggle/input/adresso-audio-data/CN", "/kaggle/input/adresso-audio-data/MCI"]

#  # Iterate through audio directories
# for directory, label in zip(audio_directories, ["AD", "CN", "MCI"]):
# # Iterate through audio files in each directory
#     output_file = f"output_{label}.csv"
#      # Open the CSV file for writing
#     with open(output_file, 'w', newline='') as csvfile:
#         csv_writer = csv.writer(csvfile)
#         csv_writer.writerow(["Filename", "Transcription", "Label"])
#         #iterate through the audio files in each directory
#         for m in os.listdir(directory):
#             audio_path = os.path.join(directory, m)
#             # Display Audio
#             audio_display = Audio(audio_path)
#             IPython.display.display(audio_display) 
#             # Perform Trancription
#             transcription_dict = model.transcribe(audio_path)
#             transcription_text = transcription_dict['text']
    
#             # Write the filename and transcription to the CSV file
#             csv_writer.writerow([m, transcription_text, label])
    
#             #get Segements and Number of words
#             segments = transcription_dict["segments"]
#             num_words = len(transcription_text.split())
#             print(f"Transcription for {m}: {transcription_text}\n")
    
#             #Calculate speech rate
    
#             audio_info = sf.info(audio_path)
#             duration = audio_info.duration
#             sample_rate = audio_info.samplerate
#             frame_duration = 0.01
#             num_frames = int(duration * sample_rate / frame_duration)
    
#             #The formula calculates the average number of words spoken per second 
#             #based on the number of words in the transcription and the duration of the audio file.
#             speech_rate = num_words / (num_frames / sample_rate)
#             print("Speech Rate (WPS):", speech_rate)
    
    
#             #Lexical diversity using TTR 
#             ttr_score = calculate_ttr(transcription_text)
#             print("TTR Score:", ttr_score)
    
#             #Lexical diversity using sdi 
#             sdi_score = calculate_sdi(transcription_text)
#             print("SDI Score:", sdi_score)
    
#             #Pause Rate 
#             pauses = detect_pauses(audio_path,sr=sample_rate)
#             print("Pause durations:", pauses)
    
#             #Syntactic Complexity:
#             avg_sentence_length, avg_clauses_per_sentence = calculate_syntactic_complexity(transcription_text)
#             print("Average Sentence Length:", avg_sentence_length)
#             print("Average Clauses per Sentence:", avg_clauses_per_sentence)

# Adding new data Pitt

# Performing Transcription and adding 4AD metrics on a data Folder 

In [ ]:
# Set the folder path containing the audio files
audio_folder = 'D:\GraduationProject#2\OpenAi\Audio_pitt_AD'

# Set the label for all audio files in the folder
label = "AD"

# Set the output CSV file
output_file = f"output_{label}.csv"

# Open the CSV file for writing
with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Filename", "Transcription", "Label", "Speech Rate", "Pause Rate", "Lexical Diversity(SDI Score)", "Syntactic Complexity(Avg Clauses per)"])

    # Get the list of audio files in the folder
    audio_files = glob.glob(os.path.join(audio_folder, "*.mp3"))

    # Loop through each audio file
    for audio_file in audio_files:
        # Display Audio
        audio_display = Audio(audio_file)
        # IPython.display.display(audio_display)
        
        # Perform Transcription
        transcription_dict = model.transcribe(audio_file)
        transcription_text = transcription_dict['text']
    
    #perform 4 features of alzheimers
    #get Segements and Number of words
        segments = transcription_dict["segments"]
        num_words = len(transcription_text.split())
    #print(f"Transcription for {m}: {transcription_text}\n")
    
    #Calculate speech rate
    
        audio_info = sf.info(audio_file)
        duration = audio_info.duration
        sample_rate = audio_info.samplerate
        frame_duration = 0.01
        num_frames = int(duration * sample_rate / frame_duration)
    
    #The formula calculates the average number of words spoken per second 
    #based on the number of words in the transcription and the duration of the audio file.
        speech_rate = num_words / (num_frames / sample_rate)
        print("Speech Rate (WPS):", speech_rate)
    
    
    #Lexical diversity using TTR 
        ttr_score = calculate_ttr(transcription_text)
        print("TTR Score:", ttr_score)
    
    #Lexical diversity using sdi 
        sdi_score = calculate_sdi(transcription_text)
        print("SDI Score:", sdi_score)
    
    #Pause Rate 
        pauses = detect_pauses(audio_file,sr=sample_rate)
        print("Pause:", pauses)
    
    #Syntactic Complexity:
        avg_sentence_length, avg_clauses_per_sentence = calculate_syntactic_complexity(transcription_text)
        print("Average Sentence Length:", avg_sentence_length)
        print("Average Clauses per Sentence:", avg_clauses_per_sentence)
        

        # Write the filename, transcription, and features to the CSV file
        csv_writer.writerow([os.path.basename(audio_file), transcription_text, label, speech_rate, pauses, sdi_score, avg_clauses_per_sentence])


# Print completion message
print("Processing complete.")

# Performing transcription and 4 AD metrics on a single file

In [26]:
audio_file = "D:\\GraduationProject#2\\OpenAi\\Audio_pitt_AD\\audioaudio.MP3"
label = "AD"
output_file = f"output_{label}.csv"

# Open the CSV file for writing
with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Filename", "Transcription", "Label", "Speech Rate", "Pause Rate", "Lexical Diversity(SDI Score)", "Syntactic Complexity(Avg Clauses per)"])

    # Display Audio
    audio_display = Audio(audio_file)
    IPython.display.display(audio_display)

    # Perform Transcription
    transcription_dict = model.transcribe(audio_file)
    transcription_text = transcription_dict['text']
    print(transcription_text)

#perform 4 features of alzheimers
    #get Segements and Number of words
    segments = transcription_dict["segments"]
    num_words = len(transcription_text.split())
    #print(f"Transcription for {m}: {transcription_text}\n")
    
    #Calculate speech rate
    
    audio_info = sf.info(audio_file)
    duration = audio_info.duration
    sample_rate = audio_info.samplerate
    frame_duration = 0.01
    num_frames = int(duration * sample_rate / frame_duration)
    
    #The formula calculates the average number of words spoken per second 
    #based on the number of words in the transcription and the duration of the audio file.
    speech_rate = num_words / (num_frames / sample_rate)
    print("Speech Rate (WPS):", speech_rate)
    
    
    #Lexical diversity using TTR 
    ttr_score = calculate_ttr(transcription_text)
    print("TTR Score:", ttr_score)
    
    #Lexical diversity using sdi 
    sdi_score = calculate_sdi(transcription_text)
    print("SDI Score:", sdi_score)
    
    #Pause Rate 
    pauses = detect_pauses(audio_file,sr=sample_rate)
    print("Pause:", pauses)
    
    #Syntactic Complexity:
    avg_sentence_length, avg_clauses_per_sentence = calculate_syntactic_complexity(transcription_text)
    print("Average Sentence Length:", avg_sentence_length)
    print("Average Clauses per Sentence:", avg_clauses_per_sentence)
        

        # Write the filename, transcription, and features to the CSV file
    csv_writer.writerow([os.path.basename(audio_file), transcription_text, label, speech_rate, pauses, sdi_score, avg_clauses_per_sentence])


# Print completion message
print("Processing complete.")


c:\Users\vip\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 This is the picture. Just tell me everything that you see happening in the future. There's a young boy getting a cookie jar and it's a bad shape because the thing is falling over and in the picture the mother is washing dishes and doesn't see it and so the water is overflowing in the sink and the dishes might get fall over and fell fall over there to get it and it's a picture of a kitchen window when the curtains are very extinct. But the water is still flowing. Okay, thank you very much.
Speech Rate (WPS): 0.017394033866105052
TTR Score: 0.6145833333333334
SDI Score: 0.9674479166666666
Pause: 28433.655172413793
2023-06-15 12:59:10,792 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
Average Sentence Length: 0.1947261663286004
Average Clauses per Sentence: 0.